# 10. 이미지 변형
이미지 전처리의 기초

In [2]:
import cv2

## 이미지 출력 함수 정의

In [2]:
def showImage(str):
    
    cv2.imshow('My Image', str)
    
    cv2.waitKey(0) # 5초간 키 입력 대기
    cv2.destroyAllWindows() # 모든 창 닫기

    cv2.waitKey(1); cv2.waitKey(1); cv2.waitKey(1); cv2.waitKey(1) # Windows 운영체제에서는 생략

## 이미지 변형(흑백)

In [5]:
img = cv2.imread('Data/poker.jpg')

showImage(img)

In [6]:
img = cv2.imread('Data/img.jpeg')
new = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
showImage(new)

## 이미지 변형(흐림)
1. Gaussian Blur 기법 사용
1. 외곽선 검출 단순화 효과   
1. 노이즈 제거 효과

### 커널 사이즈 변화에 따른 흐림 처리

In [6]:
img = cv2.imread('Data/img.jpeg')

# 커널 사이즈 값은 양수 홀수로 지정, sigmaX(가우시안 커널의 x 방향 표준편차) 값을 0으로 지정
blur = cv2.GaussianBlur(img, (21, 21), 0)

showImage(blur)

### 표준편차 변화에 따른 흐림 처리

In [7]:
img = cv2.imread('Data/img.jpeg')

# 커널 사이즈 값을 (0, 0)으로 지정, sigmaX 값을 양수로 지정
blur = cv2.GaussianBlur(img, (0, 0), 10)

showImage(blur)

## 이미지 변형(원근)
#### 사용 예시
1. 사진으로 찍은 문서를 완전한 직사각형 스캔본 형태로 변환
1. 비스듬히 찍은 사진을 정면에서 찍은 형태로 변환

### 사다리꼴 형태의 이미지를 직사각형 형태로 펼치기

In [7]:
import numpy as np

img = cv2.imread('Data/news.jpg')

width, height = 640, 240

# 순서는 좌상단부터 시계방향
src = np.array([[1678, 1178], [3378, 1157], [3686, 1835], [1514, 1860]], dtype = np.float32)
new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환

showImage(result)
write = cv2.imwrite('Data/scan.jpg', result)

### 기울어진 이미지를 수직 방향으로 세우기

In [4]:
import numpy as np

img = cv2.imread('Data/poker.jpg')

width, height = 530, 710

# 순서는 좌상단부터 시계방향
src = np.array([[2198, 460], [3525, 1299], [2242, 3115], [895, 2182]], dtype = np.float32)
new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환

showImage(result)
write = cv2.imwrite('Data/rotate.jpg', result)

## 반자동 문서 스캐너 만들기

### 마우스 이벤트 등록(Windows)

In [3]:
def mouse_handler(event, x, y, flags, param):
    
    if event == cv2.EVENT_LBUTTONDOWN:
        print('Left Button Pressed')
    elif event == cv2.EVENT_LBUTTONUP:
        print('Left Button Released')

img = cv2.imread('Data/poker.jpg')
cv2.namedWindow('My_Image')   # My_Image라는 이름의 윈도우 생성. 이 윈도우에 마우스 이벤트 처리를 위한 핸들러 적용\
cv2.setMouseCallback('My_Image', mouse_handler)

cv2.imshow('My Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 마우스 이벤트 등록(Mac)

In [ ]:
def closeImage():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

def mouseHandler(event, x, y, flags, param):
    
    if event==cv2.EVENT_LBUTTONDOWN:
        print('Left Button Pressed')
    elif event == cv2.EVENT_LBUTTONUP:
        print('Left Button Released')
    elif event == cv2.EVENT_LBUTTONDBLCLK:
        print('Left Button Double-Clicked')
        
img = cv2.imread('Data/poker.jpg')

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)

while True:
    cv2.imshow('My_Image', img)
    waitKey = cv2.waitKey(10000) & 0xFF
    if waitKey == ord('q'):
        break

closeImage()

## 프로젝트 코드(Windows)

In [ ]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 클릭한 지점 마킹하기
    for point in pointList:
        cv2.circle(img, point, 30, (255, 0, 255), cv2.FILLED)
    
    # 4개 지점이 모두 등록되면 스캔 결과 출력 및 저장  
    if len(pointList) == 4:
        showResult()

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)
cv2.imshow('My_Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 프로젝트 코드(Mac)

In [ ]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 이미지 창 종료 함수
def closeImage():
    cv2.destroyAllWindows()
    cv2.waitKey(1); cv2.waitKey(1); cv2.waitKey(1)

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 클릭한 지점 마킹하기
    for point in pointList:
        cv2.circle(img, point, 30, (255, 0, 255), cv2.FILLED)
    
    # 4개 지점이 모두 등록되면 스캔 결과 출력 및 저장  
    if len(pointList) == 4:
        showResult()

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)

while True:
    cv2.imshow('My_Image', img)
    waitKey = cv2.waitKey(10000) & 0xFF
    if waitKey == ord('q'):
        break

closeImage()                              # 이미지 창 종료

## 프로젝트 코드(버전 2, Windows)

In [ ]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 클릭한 지점 마킹하기
    for point in pointList:
        cv2.circle(img, point, 30, (255, 0, 255), cv2.FILLED)
    
    # 클릭한 두 지점을 선으로 연결
    i = len(pointList)
    if i > 1:
        cv2.line(img, pointList[i - 2], pointList[i - 1], (255, 0, 255), 10, cv2.LINE_AA)
        if i == 3:
            cv2.line(img, pointList[i - 2], pointList[0], (255, 0, 255), 10, cv2.LINE_AA)
    
    # 4개 지점이 모두 등록되면 스캔 결과 출력 및 저장  
    if len(pointList) == 4:
        showResult()

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)
cv2.imshow('My_Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 프로젝트 코드(버전 2, Mac)

In [10]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 이미지 창 종료 함수
def closeImage():
    cv2.destroyAllWindows()
    cv2.waitKey(1); cv2.waitKey(1); cv2.waitKey(1)

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 마지막으로 클릭한 지점을 저장할 변수
    prevPoint = None
    
    # 클릭한 지점 마킹하고 이전에 마지막으로 클릭한 지점과 연결하기
    for point in pointList:
        
        cv2.circle(img, point, 30, (255, 0, 255), cv2.FILLED)
        
        if prevPoint:
            cv2.line(img, prevPoint, point, (255, 0, 255), 10, cv2.LINE_AA)
        
        prevPoint = point
    
    # 4개의 꼭짓점이 모두 등록되면 결과 출력 및 스캔
    if len(pointList) == 4:
        showResult()

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)

while True:
    cv2.imshow('My_Image', img)
    waitKey = cv2.waitKey(10000) & 0xFF
    if waitKey == ord('q'):
        break

closeImage()                              # 이미지 창 종료

## 프로젝트 코드(버전 3, Windows)

In [ ]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    copyImg = img.copy()
    
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 마지막으로 클릭한 지점을 저장할 변수
    prevPoint = None
    
    # 클릭한 지점 마킹하고 이전에 마지막으로 클릭한 지점과 연결하기
    for point in pointList:
        
        cv2.circle(copyImg, point, 30, (255, 0, 255), cv2.FILLED)
        
        if prevPoint:
            cv2.line(copyImg, prevPoint, point, (255, 0, 255), 10, cv2.LINE_AA)
        
        prevPoint = point
    
    if prevPoint:
        nextPoint = (x, y)
        
        # 4개의 꼭짓점이 모두 등록되면 결과 출력 및 스캔
        if len(pointList) == 4:
            showResult()
            nextPoint = pointList[0]
        
        cv2.line(copyImg, prevPoint, nextPoint, (255, 0, 255), 10, cv2.LINE_AA)
    
    cv2.imshow('My_Image', copyImg)

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)
cv2.imshow('My_Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 프로젝트 코드(버전 3, Mac)

In [ ]:
import cv2
import numpy as np

pointList = []                              # 클릭한 좌표를 저장할 리스트
img = cv2.imread('Data/poker.jpg')

# 이미지 창 종료 함수
def closeImage():
    cv2.destroyAllWindows()
    cv2.waitKey(1); cv2.waitKey(1); cv2.waitKey(1)

# 마우스 이벤트 등록 함수
def mouseHandler(event, x, y, flags, param):
    copyImg = img.copy()
    
    # 마우스 이벤트 등록
    if event == cv2.EVENT_LBUTTONDOWN:
        pointList.append((x, y))
    
    # 마지막으로 클릭한 지점을 저장할 변수
    prevPoint = None
    
    # 클릭한 지점 마킹하고 이전에 마지막으로 클릭한 지점과 연결하기
    for point in pointList:
        
        cv2.circle(copyImg, point, 30, (255, 0, 255), cv2.FILLED)
        
        if prevPoint:
            cv2.line(copyImg, prevPoint, point, (255, 0, 255), 10, cv2.LINE_AA)
        
        prevPoint = point
    
    if prevPoint:
        nextPoint = (x, y)
        
        # 4개의 꼭짓점이 모두 등록되면 결과 출력 및 스캔
        if len(pointList) == 4:
            showResult()
            nextPoint = pointList[0]
        
        cv2.line(copyImg, prevPoint, nextPoint, (255, 0, 255), 10, cv2.LINE_AA)
    
    cv2.imshow('My_Image', copyImg)

# 사진 스캔
def showResult():
    width, height = 530, 710
    src = np.float32(pointList)
    new = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype = np.float32)

    matrix = cv2.getPerspectiveTransform(src, new)   # src 행렬을 new 행렬의 규격에 맞게 변환
    result = cv2.warpPerspective(img, matrix, (width, height))   # 변환된 행렬대로 이미지 변환
    cv2.imshow('Result', result)
    cv2.imwrite('Data/Poker_Scan.jpg', result)

cv2.namedWindow('My_Image')
cv2.setMouseCallback('My_Image', mouseHandler)

while True:
    cv2.imshow('My_Image', img)
    waitKey = cv2.waitKey(10000) & 0xFF
    if waitKey == ord('q'):
        break

closeImage()                              # 이미지 창 종료